In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import json
import numpy as np
import pickle
from struct import unpack
from base64 import b64decode
from functools import partial
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch.nn as nn

from imblearn.over_sampling import SMOTE

from common import models

Using TensorFlow backend.


In [3]:
IMG_LEN = 1024
TXT_LEN = 300
N_TOPICS = 50
N_WORTHINESSES = 2
BATCH_SIZE=2048

## Loading data

In [4]:
x_img, x_txt, y_topic, y_worthiness = pickle.load(open('unpacked_worth_topics.pickle', 'rb'))

## Preparing data

In [5]:
x_img_train, x_img_test, x_txt_train, x_txt_test, y_topic_train, y_topic_test, y_worthiness_train, y_worthiness_test = train_test_split(
    x_img, 
    x_txt, 
    y_topic,
    y_worthiness,
    test_size=0.2, 
    random_state=42,
    stratify=y_topic
)

x_img_train, x_img_val, x_txt_train, x_txt_val, y_topic_train, y_topic_val, y_worthiness_train, y_worthiness_val = train_test_split(
    x_img_train,
    x_txt_train,
    y_topic_train,
    y_worthiness_train,
    test_size=0.2,
    random_state=42,
    stratify=y_topic_train
)

In [6]:
img_sscaler = StandardScaler()
img_sscaler.fit(x_img_train)

x_img_train = img_sscaler.transform(x_img_train)
x_img_val = img_sscaler.transform(x_img_val)
x_img_test = img_sscaler.transform(x_img_test)

txt_sscaler = StandardScaler()
txt_sscaler.fit(x_txt_train)

x_txt_train = txt_sscaler.transform(x_txt_train)
x_txt_val = txt_sscaler.transform(x_txt_val)
x_txt_test = txt_sscaler.transform(x_txt_test)

In [7]:
x_img_train_t = torch.tensor(x_img_train).float()
x_img_val_t = torch.tensor(x_img_val).float()
x_img_test_t = torch.tensor(x_img_test).float()

x_txt_train_t = torch.tensor(x_txt_train).float()
x_txt_val_t = torch.tensor(x_txt_val).float()
x_txt_test_t = torch.tensor(x_txt_test).float()

y_topic_train_t = torch.tensor(y_topic_train).float()
y_topic_val_t = torch.tensor(y_topic_val).float()
y_topic_test_t = torch.tensor(y_topic_test).float()

y_worthiness_train_t = torch.tensor(y_worthiness_train).float()
y_worthiness_val_t = torch.tensor(y_worthiness_val).float()
y_worthiness_test_t = torch.tensor(y_worthiness_test).float()

In [8]:
train_ds = TensorDataset(x_img_train_t, x_txt_train_t, y_topic_train_t, y_worthiness_train_t)
val_ds = TensorDataset(x_img_val_t, x_txt_val_t, y_topic_val_t, y_worthiness_val_t)
test_ds = TensorDataset(x_img_test_t, x_txt_test_t, y_topic_test_t, y_worthiness_test_t)

In [9]:
BATCH_SIZE = 2048

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

### SMOTE upsample

In [10]:
x_train_combined = np.concatenate((x_img_train, x_txt_train), axis=1)

y_topic_train_non_categorical = np.argmax(y_topic_train, axis=1).reshape(-1, 1)
y_worthiness_train_non_categorical = np.argmax(y_worthiness_train, axis=1).reshape(-1, 1)
y_train_combined = np.concatenate(
    (y_topic_train_non_categorical, y_worthiness_train_non_categorical * 50),
    axis=1
).sum(axis=1)

In [11]:
combined_classes, combined_classes_counts = np.unique(y_train_combined, return_counts=True)

In [12]:
print('should be replaced objects with combined classses:')
for combined_class, count in zip(combined_classes, combined_classes_counts):
    if count <= 6:
        print(combined_class, '(', count,' objects)')

should be replaced objects with combined classses:
85 ( 2  objects)
92 ( 1  objects)


In [13]:
indices_to_remove_85 = np.where(y_train_combined == 85)[0]
indices_to_remove_92 = np.where(y_train_combined == 92)[0]

In [14]:
indices_to_remove = np.concatenate((indices_to_remove_85, indices_to_remove_92))

In [15]:
x_train_combined_cleaned = np.delete(x_train_combined, indices_to_remove, axis=0)
y_train_combined_cleaned = np.delete(y_train_combined, indices_to_remove, axis=0)

In [16]:
smote = SMOTE(random_state=42)
x_train_combined_smoted, y_train_combined_smoted = smote.fit_resample(
    x_train_combined_cleaned, 
    y_train_combined_cleaned
)

In [17]:
x_img_train_smoted = x_train_combined_smoted[:, :IMG_LEN]
x_txt_train_smoted = x_train_combined_smoted[:, IMG_LEN:]

y_topic_train_smoted = y_train_combined_smoted % N_TOPICS
y_worthiness_train_smoted = y_train_combined_smoted // N_TOPICS

In [18]:
print('x_img_train_smoted shape', x_img_train_smoted.shape)
print('x_txt_train_smoted shape', x_txt_train_smoted.shape)

print('\ny_topic_train_smoted shape', y_topic_train_smoted.shape)
print('y_topic_train_smoted unique', np.unique(y_topic_train_smoted))

print('\ny_worthiness_train_smoted shape', y_worthiness_train_smoted.shape)
print('y_worthiness_train_smoted unique', np.unique(y_worthiness_train_smoted))

x_img_train_smoted shape (743624, 1024)
x_txt_train_smoted shape (743624, 300)

y_topic_train_smoted shape (743624,)
y_topic_train_smoted unique [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]

y_worthiness_train_smoted shape (743624,)
y_worthiness_train_smoted unique [0 1]


In [19]:
y_topic_train_smoted_categorical = np.eye(N_TOPICS)[y_topic_train_smoted]
y_worthiness_train_smoted_categorical = np.eye(N_WORTHINESSES)[y_worthiness_train_smoted]

In [20]:
print(y_worthiness_train_smoted_categorical.shape)

(743624, 2)


### wrapping in pytorch dataset format

In [21]:
x_img_train_smoted_t = torch.tensor(x_img_train_smoted).float()
x_txt_train_smoted_t = torch.tensor(x_txt_train_smoted).float()
y_topic_train_smoted_t = torch.tensor(y_topic_train_smoted_categorical).float()
y_worthiness_train_smoted_t = torch.tensor(y_worthiness_train_smoted_categorical).float()  

In [22]:
print(y_topic_train_smoted_t.shape)

torch.Size([743624, 50])


In [23]:
print(y_topic_train_t.shape)

torch.Size([135611, 50])


In [24]:
train_ds_smoted = TensorDataset(
    x_img_train_smoted_t, 
    x_txt_train_smoted_t, 
    y_topic_train_smoted_t, 
    y_worthiness_train_smoted_t
)

In [25]:
train_loader_smoted = DataLoader(train_ds_smoted, batch_size=BATCH_SIZE)

## Learning on SMOTE upsampled dataset

In [26]:
model = models.MultitargetTridentModelBN(d=128, drop=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)
writer = SummaryWriter('runs/mt_trident_bn_d128_drop05_wd0005_smoted')

models.fit_multitarget_trident_model(
    model=model,
    optimizer=optimizer,
    epochs=100,
    writer=writer,
    train_loader=train_loader_smoted,
    val_loader=val_loader
)

epoch: 0 train_loss: tensor(12.0629, grad_fn=<AddBackward0>) average train loss tensor(11.8758, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.09214523788455299 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(13.5351) 
worthiness_roc_auc_score: 0.5299411322331284 
worthiness_pr_auc_score: 0.5126389994985694
epoch: 1 train_loss: tensor(12.0246, grad_fn=<AddBackward0>) average train loss tensor(11.9530, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.18508686546913253 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(12.4225) 
worthiness_roc_auc_score: 0.47363391159919865 
worthiness_pr_auc_score: 0.5126389994985694
epoch: 2 train_loss: tensor(12.2301, grad_fn=<AddBackward0>) average train loss tensor(11.8999, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.24269238710438604 
val_worthiness_common_acc: 0.02878801286021886 
val_avg_loss: tensor(11.7932) 
worthiness_roc_auc_score: 0.509476889150875 
worthiness_pr_auc_score: 0.5103525788746539

epoch: 25 train_loss: tensor(11.5482, grad_fn=<AddBackward0>) average train loss tensor(11.7149, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.2713329203905259 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(11.7291) 
worthiness_roc_auc_score: 0.4692867306143955 
worthiness_pr_auc_score: 0.5126389994985694
epoch: 26 train_loss: tensor(11.5227, grad_fn=<AddBackward0>) average train loss tensor(11.7047, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.26021296050497006 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(11.7751) 
worthiness_roc_auc_score: 0.47854227772453395 
worthiness_pr_auc_score: 0.5126389994985694
epoch: 27 train_loss: tensor(11.5269, grad_fn=<AddBackward0>) average train loss tensor(11.7055, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.2550806713270212 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(11.7893) 
worthiness_roc_auc_score: 0.47641867765953483 
worthiness_pr_auc_score: 0.512638999498

epoch: 50 train_loss: tensor(11.4296, grad_fn=<AddBackward0>) average train loss tensor(11.6799, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.29106568740229477 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(11.5830) 
worthiness_roc_auc_score: 0.4794114914671822 
worthiness_pr_auc_score: 0.5126389994985694
epoch: 51 train_loss: tensor(11.4413, grad_fn=<AddBackward0>) average train loss tensor(11.6724, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.28389818010205586 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(11.6424) 
worthiness_roc_auc_score: 0.4915882521100851 
worthiness_pr_auc_score: 0.5126389994985694
epoch: 52 train_loss: tensor(11.4056, grad_fn=<AddBackward0>) average train loss tensor(11.6783, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.26637760670147187 
val_worthiness_common_acc: 0.025277998997138895 
val_avg_loss: tensor(11.6601) 
worthiness_roc_auc_score: 0.48119133782681633 
worthiness_pr_auc_score: 0.51263899949

epoch: 75 train_loss: tensor(11.3545, grad_fn=<AddBackward0>) average train loss tensor(11.6428, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.2924224994838215 
val_worthiness_common_acc: 0.02598590095271805 
val_avg_loss: tensor(11.6561) 
worthiness_roc_auc_score: 0.4917091366082045 
worthiness_pr_auc_score: 0.5126479530092387
epoch: 76 train_loss: tensor(11.3746, grad_fn=<AddBackward0>) average train loss tensor(11.6493, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.29469368492463793 
val_worthiness_common_acc: 0.027077249800902575 
val_avg_loss: tensor(11.5667) 
worthiness_roc_auc_score: 0.4804261920955839 
worthiness_pr_auc_score: 0.5120790715227618
epoch: 77 train_loss: tensor(11.3647, grad_fn=<AddBackward0>) average train loss tensor(11.6474, grad_fn=<DivBackward0>)
val_topics_common_acc: 0.3013302657581925 
val_worthiness_common_acc: 0.03144264519364068 
val_avg_loss: tensor(11.5384) 
worthiness_roc_auc_score: 0.4849048506409968 
worthiness_pr_auc_score: 0.512134629744663


In [28]:
predictions_topics = torch.tensor([])
predictions_worthiness = torch.tensor([])

with torch.no_grad():
    for x_img_cur, x_txt_cur, _, _ in test_loader:
        out_topics,_, _, out_worthiness = model(x_img_cur.float(), x_txt_cur.float())
        predictions_topics = torch.cat((predictions_topics, out_topics), 0)
        predictions_worthiness = torch.cat((predictions_worthiness, out_worthiness), 0)

predictions_topics = predictions_topics.numpy()
predictions_worthiness = predictions_worthiness.numpy()

print(classification_report(np.argmax(y_topic_test, axis=1), np.argmax(predictions_topics, axis=1)))
print(classification_report(np.argmax(y_worthiness_test, axis=1), np.argmax(predictions_worthiness, axis=1)))

print('topics roc auc score:', roc_auc_score(y_topic_test, predictions_topics))
print('worthiness roc auc score:', roc_auc_score(y_worthiness_test, predictions_worthiness))

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.83      0.25      0.39      1694
           1       1.00      0.00      0.01      1512
           2       0.00      0.00      0.00       213
           3       0.92      0.37      0.53      1736
           4       0.58      0.05      0.09       295
           5       0.49      0.14      0.22       682
           6       0.91      0.71      0.80      1082
           7       0.66      0.20      0.30       543
           8       0.49      0.06      0.11       501
           9       0.84      0.36      0.50       903
          10       0.87      0.72      0.79      1307
          11       0.93      0.27      0.42      1130
          12       0.62      0.13      0.21      1241
          13       0.72      0.54      0.62      1007
          14       0.45      0.13      0.20       816
          15       0.45      0.21      0.29       514
          16       0.79      0.54      0.64      1851
          17       0.14    